In [32]:
import pyprind
import pandas as pd
import os
from nltk.corpus import stopwords
import re
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from keras.layers import Embedding, LSTM, GRU, Dropout, Dense, Input
from keras.models import Model, Sequential, load_model
from keras.preprocessing import sequence
from keras.datasets import imdb
import gensim
from gensim.models.word2vec import Word2Vec
from keras.layers import Input, Flatten, Dropout, Dense, Embedding, Conv1D, MaxPooling1D, Activation
from keras.models import Model, Sequential
from keras.layers.normalization import BatchNormalization
from keras.optimizers import Adam
from keras.layers.recurrent import LSTM

In [ ]:
import nltk
nltk.download('stopwords') #下載stopwords辭典

In [29]:
def tokenizer(text): #將完整語句切割成個別單字，剃除停止詞
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized

In [35]:
stop = stopwords.words('english') 
basepath = r'D:\jupyter\training_dataset_v1\training_dataset_v1\training_dataset'


In [22]:
labels = {'pos': 1, 'neg': 0}
pbar = pyprind.ProgBar(50000)
df = pd.DataFrame()
#每個檔案彙整，切割成個別單字後。全部儲存成csv檔案，方便之後資料IO

for l in ('pos', 'neg'):
    path = os.path.join(basepath, l)
    for file in os.listdir(path):
        with open(os.path.join(path, file), 'r', encoding='utf-8') as infile:
            txt = infile.read()
            token = tokenizer(text=txt)
        df = df.append([[token, labels[l]]], ignore_index=True)
        pbar.update()
df.columns = ['review', 'sentiment']
np.random.seed(0)
df = df.reindex(np.random.permutation(df.index))
df.to_csv('movie_data.csv') 


KeyboardInterrupt: 

In [ ]:
import pyprind
import gensim.models
import re

inpath = 'movie_data.csv'
outpath = 'wordVectTrainResult'
pbar = pyprind.ProgBar(100000)
class csvStream(object):
    def __init__(self,path):
        self.path=path
    def __iter__(self):
        with open(self.path, 'r',encoding="utf-8") as csv:
            next(csv)  # skip header
            for line in csv:
                text = line[4:-3]
                text = re.sub('[\'\"\[\]\d\b]','',text)   
                while (text[0] == ',') or (text[0] == ' '):
                    text = text[1:]
                pbar.update()
                yield text.split(', ')

#製作字典，資料以評論所出現過的單字進行整理，並依照出現頻率由大到小排序
lineIterator = csvStream(inpath)
model = gensim.models.Word2Vec()
model.build_vocab(lineIterator)
print('vocabulary building finished, start training...')
model.train(lineIterator,total_examples=model.corpus_count,epochs=20)
model.save(outpath)

In [2]:
#讀取字典
inpath = 'wordVectTrainResult'
model = gensim.models.Word2Vec.load(inpath)

In [4]:
#建立轉換資料矩陣，以及餵入模型中的鑲嵌層
def build_word2idx_embedMatrix(w2vModel):
    word2idx = {"_stopWord": 0}  # 这里加了一行是用来过滤停用词的。
    vocab_list = [(w, w2vModel.wv[w]) for w, v in w2vModel.wv.vocab.items()]
    embedMatrix = np.zeros((len(w2vModel.wv.vocab.items()) + 1, w2vModel.vector_size))#+1是因为停用词
    for i in range(0, len(vocab_list)):
        word = vocab_list[i][0]#vocab_list[i][0]是词
        word2idx[word] = i + 1#因为加了停用词，所以其他索引都加1
        embedMatrix[i + 1] = vocab_list[i][1]#vocab_list[i][1]这个词对应embedding_matrix的那一行
    return word2idx, embedMatrix

In [4]:
word2idx, embedMatrix = build_word2idx_embedMatrix(model)

In [28]:
import keras

In [63]:
#神經網路模型
sequence_input = Input(shape=(500,), dtype=np.int32)
embedding_layer = Embedding(input_dim=len(embedMatrix), output_dim=len(embedMatrix[0]),
                                weights=[embedMatrix],  # 表示直接使用预训练的词向量
                                trainable=False)
embedded_sequences = embedding_layer(sequence_input)
x = Dropout(.25)(embedded_sequences)
x = Conv1D(64, 5, padding='same', activation='relu')(x)
x = Dropout(.25)(x)
x = MaxPooling1D(5)(x)
x = Flatten()(x)
x = Dense(100, activation='relu')(x)
x = Dropout(.7)(x)
y = Dense(1, activation='sigmoid')(x)
y = keras.layers.Activation(activation=keras.activations.sigmoid,name="sigmoe")(y)
model = Model(sequence_input, y)
model.summary()
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['acc']
)

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 500)               0         
_________________________________________________________________
embedding_7 (Embedding)      (None, 500, 100)          2861300   
_________________________________________________________________
dropout_10 (Dropout)         (None, 500, 100)          0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, 500, 64)           32064     
_________________________________________________________________
dropout_11 (Dropout)         (None, 500, 64)           0         
_________________________________________________________________
max_pooling1d_4 (MaxPooling1 (None, 100, 64)           0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 6400)              0   

In [3]:
#將文字矩陣，轉換成數字矩陣，取每個資料中，在字典出現頻率最高前500字
def make_deepLearn_data(sentenList, word2idx):
    X_train_idx = [[word2idx.get(w, 0) for w in sen] for sen in sentenList]#之前都是通过word处理的，这里word2idx讲word转为id
    X_train_idx = np.array(sequence.pad_sequences(X_train_idx, maxlen=500))  # padding成相同长度
    return X_train_idx


In [2]:
#整理資料
def stream_docs(path):
    with open(path, 'r',encoding="utf-8") as csv:
        next(csv)  # skip header
        for line in csv:
            text, label = line[4:-3], int(line[-2])
            text = re.sub('[\'\"\[\]\d\b]','',text)
            while text[0] == ',':
                    text = text[1:]
            yield text.split(', '), label

#取出特定筆數資料
def get_minibatch(doc_stream, size):
    docs, y = [], []
    try:
        for _ in range(size):
            text, label = next(doc_stream)
            docs.append(text)
            y.append(label)
    except StopIteration:
        return None, None
    return docs, y

In [8]:
#訓練23000筆 驗證2000筆
BatchSize = 1000
BatchNum = 23
doc_stream = stream_docs(path='movie_data.csv') 
X_raw, y_train = get_minibatch(doc_stream, size=BatchSize*BatchNum)
X_raw_test, y_test = get_minibatch(doc_stream, size=(25000-BatchNum*BatchSize))

In [9]:
#文字矩陣轉數字矩陣，才可讓模型訓練
x_data = make_deepLearn_data(X_raw,word2idx)
x_test = make_deepLearn_data(X_raw_test,word2idx)

In [10]:
y_train = np.array(y_train)
y_test = np.array(y_test)

In [64]:
model.fit(x=x_data,y=y_train,epochs=20, batch_size=100, verbose=1,validation_data=(x_test, y_test))

Train on 23000 samples, validate on 2000 samples
Epoch 1/20
23000/23000 [==============================] - 26s 1ms/step - loss: 0.6934 - acc: 0.5082 - val_loss: 0.6931 - val_acc: 0.5125
Epoch 2/20
23000/23000 [==============================] - 26s 1ms/step - loss: 0.6934 - acc: 0.5066 - val_loss: 0.6931 - val_acc: 0.5090
Epoch 3/20
23000/23000 [==============================] - 24s 1ms/step - loss: 0.6932 - acc: 0.5036 - val_loss: 0.6931 - val_acc: 0.5095
Epoch 4/20
23000/23000 [==============================] - 25s 1ms/step - loss: 0.6912 - acc: 0.5163 - val_loss: 0.6742 - val_acc: 0.6205
Epoch 5/20
23000/23000 [==============================] - 24s 1ms/step - loss: 0.6116 - acc: 0.7589 - val_loss: 0.5715 - val_acc: 0.8320
Epoch 6/20
23000/23000 [==============================] - 25s 1ms/step - loss: 0.5861 - acc: 0.8239 - val_loss: 0.5702 - val_acc: 0.8645
Epoch 7/20
23000/23000 [==============================] - 25s 1ms/step - loss: 0.5769 - acc: 0.8401 - val_loss: 0.5714 - val_acc:

In [16]:
#將模型存檔
model.save('imdb.h5')

In [1]:
from keras.models import load_model
model = load_model('imdb.h5')

Using TensorFlow backend.
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [5]:
def text_predict(filepath,word2idx,model):
    with open(filepath, 'r', encoding='utf-8') as infile:
        txt = infile.read()
        token = [tokenizer(text=txt)]
        data=make_deepLearn_data(token,word2idx)
        return(model.predict(data))
        

In [41]:
#讀取字典
inpath = 'wordVectTrainResult'
dic = gensim.models.Word2Vec.load(inpath)
word2idx, embedMatrix = build_word2idx_embedMatrix(dic)

In [ ]:
#大於0.5判為 pos 小於0.5 判為neg
text_predict(r'D:\jupyter\training_dataset_v1\training_dataset_v1\training_dataset\pos\0_9.txt',word2idx,model)

In [1]:
import re
from nltk.corpus import stopwords
import gensim
import numpy as np
from keras.preprocessing import sequence
from keras.models import load_model
import pandas as pd
import numpy as np
#建立轉換資料矩陣，以及餵入模型中的鑲嵌層
def build_word2idx_embedMatrix(w2vModel):
    word2idx = {"_stopWord": 0}  # 这里加了一行是用来过滤停用词的。
    vocab_list = [(w, w2vModel.wv[w]) for w, v in w2vModel.wv.vocab.items()]
    embedMatrix = np.zeros((len(w2vModel.wv.vocab.items()) + 1, w2vModel.vector_size))#+1是因为停用词
    for i in range(0, len(vocab_list)):
        word = vocab_list[i][0]#vocab_list[i][0]是词
        word2idx[word] = i + 1#因为加了停用词，所以其他索引都加1
        embedMatrix[i + 1] = vocab_list[i][1]#vocab_list[i][1]这个词对应embedding_matrix的那一行
    return word2idx, embedMatrix
def tokenizer(text): #將完整語句切割成個別單字，剃除停止詞
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = [w for w in text.split() if w not in stop]
    return tokenized
#將文字矩陣，轉換成數字矩陣，取每個資料中，在字典出現頻率最高前500字
def make_deepLearn_data(sentenList, word2idx):
    X_train_idx = [[word2idx.get(w, 0) for w in sen] for sen in sentenList]#之前都是通过word处理的，这里word2idx讲word转为id
    X_train_idx = np.array(sequence.pad_sequences(X_train_idx, maxlen=500))  # padding成相同长度
    return X_train_idx
stop = stopwords.words('english') 
model = load_model('imdb.h5')
#讀取字典
inpath = 'wordVectTrainResult'
dic = gensim.models.Word2Vec.load(inpath)
word2idx, embedMatrix = build_word2idx_embedMatrix(dic)

Using TensorFlow backend.
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarning: Passing (type, 1) or '1type' as

D:\anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
D:\anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
D:\anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
D:\anaconda\lib\site-packages\tensorboard\compat\tensorflow_stub\dtypes.py:544: FutureWarning: Passing (type, 1) or '1type' 

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



In [3]:
csv= pd.read_csv('test_dataset.csv',header=None,encoding="utf-8")
csv = csv[1].tolist()
test_data=[]
count=1
for line in csv:
    if str(line)=="nan":
        test_data.append("")
    else:
        test_data.append(tokenizer(line))
    count=count+1
x_test = make_deepLearn_data(test_data,word2idx)
answer = model.predict(x_test)
answer = answer.tolist()
status = [1 if value[0] >0.5 else 0 for value in answer]
df=pd.DataFrame(range(1,len(status)+1))
df.columns = ['Id']
df['Label'] = status
df.to_csv('submission.csv',index=False) 